In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [1]:
import torch
from torch import nn

In [2]:
x=torch.ones(3)

In [3]:
torch.save(x,'x.pt')

In [4]:
x2 = torch.load('x.pt')

In [5]:
x2

tensor([1., 1., 1.])

In [7]:
y = torch.zeros(4)

In [8]:
torch.save([x,y],"xy.pt")

In [9]:
xy_list = torch.load("xy.pt")

In [10]:
xy_list

[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

In [11]:
torch.save({'x':x,'y':y},"xy_dict.pt")

In [12]:
xy_dict= torch.load('xy_dict.pt')

In [13]:
xy_dict

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

In [17]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.hidden = nn.Linear(3,2)
        self.act = nn.ReLU()
        self.output=nn.Linear(2,1)
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [18]:
net = MLP()
net

MLP(
  (hidden): Linear(in_features=3, out_features=2, bias=True)
  (act): ReLU()
  (output): Linear(in_features=2, out_features=1, bias=True)
)

In [19]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0889, -0.5772,  0.4814],
                      [ 0.3075,  0.3277, -0.0864]])),
             ('hidden.bias', tensor([ 0.0183, -0.3384])),
             ('output.weight', tensor([[ 0.1203, -0.5843]])),
             ('output.bias', tensor([0.3893]))])

In [20]:
optimizer = torch.optim.SGD(net.parameters(),lr = 0.001,momentum=0.9)

In [22]:
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [0, 1, 2, 3]}]}

### 保存和加载模型

##### 仅保存和加载模型参数 `state_dict`

保存：
```python
torch.save(model.state_dict(),path)
```

加载：
```python
model = TheModelClass(*args,**kwargs)
model.load_state_dict(torch.load(PATH))
```

#### 保存和加载整个模型

保存：
```python
torch.save(model,PATH)
```

加载：
```python
model = torch.load(PATH)
```

In [26]:
X = torch.randn(2,3)

In [27]:
Y = net(X)

In [28]:
PATH = "./net.pt"

In [29]:
torch.save(net.state_dict(),PATH)

In [30]:
net2 = MLP()

In [31]:
net2.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [32]:
Y2 = net2(X)

In [35]:
(Y2==Y).int()

tensor([[1],
        [1]], dtype=torch.int32)

In [36]:
torch.cuda.is_available()

False